In [35]:
# -*- coding: utf-8 -*-
import os
import glob
import cv2
import pandas as pd
import mxnet as mx
from tqdm import tqdm

# 统计类别
# Load all txt files
aus = [1, 2, 4, 5, 6, 9, 12, 17, 20, 25, 26, 43]
names = ['image'] + ['AU{:0>2}'.format(i) for i in aus]
cols = [0] + [i + 1 for i in aus]
types = dict(zip(names, ['str'] + ['int'] * len(aus)))
start_index = 0
txt_dfs = []

# Scan all images
image_dir = '../../src_wdx/data/testset_aligned_112/'
all_images = []
sub_dirs = [name for name in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, name))]
for sub_dir in tqdm(sub_dirs, desc='Scanning images'):
    image_fnames = [name for name in os.listdir(os.path.join(image_dir, sub_dir))
                    if not name.startswith('.') and name.endswith('.jpg')]
    all_images.extend(image_fnames)
all_images = set(all_images)

# Load all txt files
path = '../../src_wdx/data/dataFile_1000.txt'
txt_df = pd.read_csv(path, sep='\t', names=names, usecols=cols, dtype=types)
print(txt_df.shape)

txt_df['image'] = txt_df['image'].str[-22:]
txt_df.index = list(range(0, txt_df.shape[0]))
txt_dfs.append(txt_df)
start_index += txt_df.shape[0]
# Concatenate and reset index
total_df = pd.concat(txt_dfs)
tqdm.write('Found {} labels in {}'.format(total_df.shape[0], path))
total_df = total_df[total_df['image'].isin(all_images)]
total_df.reset_index(drop=True, inplace=True)
total_df.to_csv('../../Data/norm_data/tongji/all_label.csv', index=False)
print(total_df.shape)
a1 = txt_df.drop_duplicates(subset=names[1:],keep='first')
a1 = a1[names[1:]]
a1.to_csv('../../Data/norm_data/tongji/classes_999.csv', index=False)
a = txt_df.replace(999, 0)
b = a.drop_duplicates(subset=names[1:],keep='first')
b = b[names[1:]]
b.to_csv('../../Data/norm_data/tongji/classes.csv', index=False)

Scanning images: 100%|██████████| 42/42 [00:00<00:00, 292.51it/s]


(24665, 13)
Found 24665 labels in ../src_wdx/data/dataFile_1000.txt
(24528, 13)


In [36]:
# -*- coding: utf-8 -*-
# 统计各类别出现的次数
import pandas as pd
import numpy as np
import mxnet as mx
import os

csv = pd.read_csv('../../Data/norm_data/tongji/classes.csv')
classes = np.array(csv)

# 读取label
aus = [1, 2, 4, 5, 6, 9, 12, 17, 20, 25, 26, 43]
names = ['AU{:0>2}'.format(i) for i in aus]
all_label = pd.read_csv('../../Data/norm_data/tongji/all_label.csv')
all_label = all_label[names]
all_label = np.array(all_label, dtype=np.int)
print(len(all_label))

C = []
for i in range(len(classes)):
    count = 0
    for j in range(len(all_label)):
        label = all_label[j]
        valid = np.not_equal(label, 999)
        label = np.bitwise_and(valid, label)
        if (label == classes[i]).all():
            count += 1
    C.append(count)
print(np.shape(C), end=' ')
new_label_names = []
for i in range(len(C)):
    temp = classes[i]
    new_label = 'AU'
    for j in range(len(temp)):
        if temp[j] == 1:
            new_label = new_label + '-' + names[j][2:]
    new_label_names.append(new_label)
    

d = pd.DataFrame(C, index=new_label_names, columns=['count'])
d.to_csv('../../Data/norm_data/tongji/all_name_count.csv')
print('Done...')

24528
(355,) Done...


In [1]:
# 统计样本大于50的类
import pandas as pd 
import numpy as np

# 读取类别文件
csv = pd.read_csv('../../Data/norm_data/tongji/classes.csv')
classes = np.array(csv)
name_csv = pd.read_csv('../../Data/norm_data/tongji/all_name_count.csv')
count = [i for i in name_csv['count']]
name = [i for i in name_csv['name']]

new_classes = []
new_name = []
for i in range(len(count)):
    if count[i]>=50:
        new_classes.append(classes[i])
        new_name.append([name[i], count[i]])
        
aus = [1, 2, 4, 5, 6, 9, 12, 17, 20, 25, 26, 43]
names = ['AU{:0>2}'.format(i) for i in aus]
new_classes = pd.DataFrame(new_classes, columns=names)
new_name = pd.DataFrame(new_name, columns=['name', 'count'])
new_classes.to_csv('../../Data/norm_data/tongji/classes_above50.csv', index=False)
new_name.to_csv('../../Data/norm_data/tongji/name_count_above50.csv', index=False)

In [49]:
# 去除大于50类中的0类
import pandas as pd 
import numpy as np

# 读取类别文件
csv = pd.read_csv('../../Data/norm_data/tongji/classes_above50.csv')
classes = np.array(csv)
name_csv = pd.read_csv('../../Data/norm_data/tongji/name_count_above50.csv')
count = [i for i in name_csv['count']]
name = [i for i in name_csv['name']]

new_classes = []
new_name = []
for i in range(len(count)):
    if sum(classes[i])>0:
        new_classes.append(classes[i])
        new_name.append([name[i], count[i]])
        
aus = [1, 2, 4, 5, 6, 9, 12, 17, 20, 25, 26, 43]
names = ['AU{:0>2}'.format(i) for i in aus]
new_classes = pd.DataFrame(new_classes, columns=names)
new_name = pd.DataFrame(new_name, columns=['name', 'count'])
new_classes.to_csv('../../Data/norm_data/tongji/classes_above50_no0.csv', index=False)
new_name.to_csv('../../Data/norm_data/tongji/name_count_above50_no0.csv', index=False)
print('Done')

Done


In [42]:
# -*- coding: utf-8 -*-
# 去除没有AU的数据
import pandas as pd
import mxnet as mx

# 读取rec
record_path = '../../Data/norm_data/rec/all.rec'
record = mx.recordio.MXIndexedRecordIO(os.path.splitext(record_path)[0] + '.idx', record_path, 'r')
indices = record.keys

out_path = '../../Data/norm_data/rec/all_no0.rec'
out_record = mx.recordio.MXIndexedRecordIO(os.path.splitext(out_path)[0] + '.idx', out_path, 'w')

num = 0
for i in range(len(indices)):
    idx = indices[i]
    item = record.read_idx(idx)
    header, img = mx.recordio.unpack_img(item)
    label = header.label
    # new_label = num_items_0
    label = np.array(label, dtype=np.int32)
    valid = np.not_equal(label, 999)
    label = np.bitwise_and(valid, label).astype(np.int32)
    if  sum(label) != 0:
        out_record.write_idx(num, item)
        num += 1
    print('\rWriting {} items to {}... '.format(num, os.path.basename(out_path)), end='')

out_record.close()
record.close()
print('Done')

Writing 16111 items to all_no0.rec... Done


In [50]:
# -*- coding: utf-8 -*-
# 读取样本数大于50个的数据作为train
import pandas as pd
import mxnet as mx

# 读取类别文件
csv = pd.read_csv('../../Data/insightface_data/tongji_version4/classes_above100.csv')
classes = np.array(csv)
name = pd.read_csv('../../Data/insightface_data/tongji_version4/name_count_above100.csv')
count = [i for i in name['count']]
print(len(count))
# 读取rec
record_path = '../../Data/insightface_data/rec_version4/all_no999.rec'
record = mx.recordio.MXIndexedRecordIO(os.path.splitext(record_path)[0] + '.idx', record_path, 'r')
indices = record.keys

out_path = '../../Data/norm_data/rec/train_above50.rec'
out_record = mx.recordio.MXIndexedRecordIO(os.path.splitext(out_path)[0] + '.idx', out_path, 'w')

num = 0
for i in range(len(indices)):
    idx = indices[i]
    item = record.read_idx(idx)
    header, img = mx.recordio.unpack_img(item)
    label = header.label
    # new_label = num_items_0
    label = np.array(label, dtype=np.int32)
    valid = np.not_equal(label, 999)
    label = np.bitwise_and(valid, label).astype(np.int32)
    for i in range(len(classes)):
        if (label == classes[i]).all():
            out_record.write_idx(num, item)
            num += 1
            break
    print('\rWriting {} items to {}... '.format(num, os.path.basename(out_path)), end='')

out_record.close()
record.close()
print('Done')

32
Writing 14282 items to train_above50_no0.rec... Done


In [51]:
# -*- coding: utf-8 -*-
# 重新编码
import os
import argparse
import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon.data.vision import transforms as T
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import normalize
import pandas as pd


csv = pd.read_csv('../../Data/norm_data/tongji/classes_above50_no0.csv')
train_classes = np.array(csv)
print(len(train_classes))

record_path = '../../Data/norm_data/rec/train_above50_no0.rec'
out_record_path = '../../Data/norm_data/rec/train_above50_newlabel.rec'

record = mx.recordio.MXIndexedRecordIO(os.path.splitext(record_path)[0] + '.idx',
                                       record_path, 'r')

indices = record.keys

out_record = mx.recordio.MXIndexedRecordIO(os.path.splitext(out_record_path)[0] + '.idx',
                                             out_record_path, 'w')

num_items_0 = 0
for i in range(len(indices)):
    idx = indices[i]
    item = record.read_idx(idx)
    header, img = mx.recordio.unpack_img(item)
    label = header.label
    # new_label = num_items_0
    label = np.array(label, dtype=np.int32)
    valid = np.not_equal(label, 999)
    label = np.bitwise_and(valid, label).astype(np.int32)
    for j in range(len(train_classes)):
        if (label == train_classes[j]).all():
            new_label = j
            break
    header = mx.recordio.IRHeader(flag=0, label=new_label, id=i, id2=0)
    s = mx.recordio.pack_img(header, img, quality=100, img_fmt='.jpg')
    out_record.write_idx(i, s)
    num_items_0 += 1
    print('\rWriting {} items to {}... '.format(num_items_0, os.path.basename(out_record_path)), end='')

out_record.close()
print('Done.')


32
Writing 14282 items to train_above50_newlabel.rec... Done.
